In [ ]:
import pathlib
import sys

In [ ]:
import numpy as np
import joblib
import pandas as pd
from IPython.display import display
from tqdm import tqdm

In [ ]:
src_dir = "../../src"

In [ ]:
sys.path.append(src_dir)

In [ ]:
from package.constants import *
from package.datasets import *
from package.feature_extraction import *

In [ ]:
processed = load_processed()

In [ ]:
model = joblib.load(lgbm_reg_path)

In [ ]:
is_test = processed[target].isnull()
dates = processed.loc[is_test, "date"].unique()

In [ ]:
# This process will need to be fixed later
processed.loc[is_test, target] = 0.0

In [ ]:
%%time
for date in tqdm(dates[:test_days]):
    is_date = (processed["date"] >= date - np.timedelta64(max_lags, "D")) & (
        processed["date"] <= date
    )

    intermediate = processed[is_date].copy()

    # See below:
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/139461
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/141515
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/141545
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/141743
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/141972
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/151339
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/151614
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/151927
    create_shift_features(intermediate, level_ids[11], [target], periods_online)
    create_rolling_features(
        intermediate, level_ids[11:], shift_features_online, windows
    )

    is_date = intermediate["date"] == date
    X = intermediate.loc[is_date, features]

    display(X.isnull().sum())

    y_pred = model.predict(X)
    y_pred = y_pred.astype("float32")
    y_pred /= intermediate.loc[is_date, "sell_price"]

    # See https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/137217
    y_pred = np.maximum(0.0, y_pred)

    # See below:
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/141452
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/142770
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/143295
    y_pred *= magic_multiplier

    is_date = processed["date"] == date
    processed.loc[is_date, target] = y_pred

In [ ]:
processed = processed[is_test]

In [ ]:
processed.groupby("date")[target].sum().plot(figsize=(16, 9), grid=True, marker="o")

In [ ]:
processed.groupby(["date", "state_id"])[target].sum().unstack().plot(
    figsize=(16, 9), grid=True, marker="o"
)

In [ ]:
processed.groupby(["date", "store_id"])[target].sum().unstack().plot(
    figsize=(16, 9), grid=True, marker="o"
)

In [ ]:
processed.groupby(["date", "cat_id"])[target].sum().unstack().plot(
    figsize=(16, 9), grid=True, marker="o"
)

In [ ]:
processed.groupby(["date", "dept_id"])[target].sum().unstack().plot(
    figsize=(16, 9), grid=True, marker="o"
)

In [ ]:
models_dir_path.mkdir(parents=True, exist_ok=True)
processed.to_parquet(prediction_path)